In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip '/content/gdrive/MyDrive/hw4/sort-master.zip'
!unzip '/content/gdrive/MyDrive/hw4/KITTI_17_images.zip'
!unzip '/content/gdrive/MyDrive/hw4/TrackEval-master.zip'

Archive:  /content/gdrive/MyDrive/hw4/sort-master.zip
 extracting: sort-master/.gitignore  
   creating: sort-master/data/
   creating: sort-master/data/train/
   creating: sort-master/data/train/ADL-Rundle-6/
   creating: sort-master/data/train/ADL-Rundle-6/det/
  inflating: sort-master/data/train/ADL-Rundle-6/det/det.txt  
   creating: sort-master/data/train/ADL-Rundle-8/
   creating: sort-master/data/train/ADL-Rundle-8/det/
  inflating: sort-master/data/train/ADL-Rundle-8/det/det.txt  
   creating: sort-master/data/train/ETH-Bahnhof/
   creating: sort-master/data/train/ETH-Bahnhof/det/
  inflating: sort-master/data/train/ETH-Bahnhof/det/det.txt  
   creating: sort-master/data/train/ETH-Pedcross2/
   creating: sort-master/data/train/ETH-Pedcross2/det/
  inflating: sort-master/data/train/ETH-Pedcross2/det/det.txt  
   creating: sort-master/data/train/ETH-Sunnyday/
   creating: sort-master/data/train/ETH-Sunnyday/det/
  inflating: sort-master/data/train/ETH-Sunnyday/det/det.txt  
   cr

In [6]:
!pip install cv
# !pip install -r sort-master/requirements.txt
import torch
import torchvision
import cv2
import sys
sys.path.insert(0,'./sort-master/')
import matplotlib
from google.colab.patches import cv2_imshow
from collections import namedtuple, OrderedDict
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
yolov5_detector = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)
yolov5_detector.float()
yolov5_detector.eval()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-5-6 Python-3.10.11 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 14.1M/14.1M [00:00<00:00, 131MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [5]:
!pip install filterpy
from sort import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110473 sha256=8b394597a525f730503e0b5ff80f3d06abfeda64b8d73510fec765e6ce49c4be
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


In [18]:
num_colors = 32
color_array = plt.cm.jet(np.linspace(0, 1, num_colors))
color_array = [(int(r * 255), int(g * 255), int(b * 255)) for r, g, b, _ in color_array]

mot_tracker = Sort()

images = sorted(glob.glob('/content/KITTI_17_images/*.jpg'))

x_dim = cv2.imread(images[0]).shape[1]
y_dim = cv2.imread(images[0]).shape[0]
out = cv2.VideoWriter('/content/output_video.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 10, (x_dim, y_dim))

det = []
results = []

for image in images:
  img = cv2.imread(image)
  yolo_predictions = yolov5_detector(img)
  detections = yolo_predictions.pred[0].numpy()

  pedestrian = []
  for i in range(len(detections)):
    if(detections[i][-1] == 0):
      pedestrian.append(detections[i])

  pedestrian = np.squeeze(pedestrian)
  det.append(pedestrian)

  track_bbs_ids = mot_tracker.update(pedestrian)
  results.append(track_bbs_ids)

  for i in range(len(track_bbs_ids)):
    loc = track_bbs_ids.tolist()[i]
    x1, y1, x2, y2 = int(loc[0]), int(loc[1]), int(loc[2]), int(loc[3])

    name = "ID: {}".format(str(int(loc[4])))

    cv2.rectangle(img, (x1, y1), (x2, y2), color_array[int(loc[4]) % 32], 4)
    
    cv2.putText(img, 
                name, 
                org = (x1, y1-5), 
                fontFace = 0,
                fontScale = 0.6,
                color = color_array[int(loc[4]) % 32],
                thickness = 2)

  out.write(img)

out.release()

import numpy as np

def process_results(results):
    processed_results = []

    for i, result in enumerate(results):
        result = np.insert(result, 0, i + 1, axis=1)
        
        result[:, 3] -= result[:, 1]
        result[:, 4] -= result[:, 2]
        
        result = result[:, [0, 5, 1, 2, 3, 4]]

        result = np.hstack([result, np.full((result.shape[0], 4), -1)])

        processed_results.append(result)

    processed_results = np.concatenate(processed_results, axis=0)

    return processed_results

processed_results = process_results(results)
np.savetxt('out.txt', processed_results)